In [1]:
import tensorflow.compat.v2 as tf
import tensorflow_datasets as tfds
from typing import Union
import matplotlib.pyplot as plt
import numpy as np
import cv2
# from keras.utils.vis_utils import plot_model
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score

# from keras_flops import get_flops
import os
import glob
import numpy as np
import keras
from keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array, array_to_img

from PIL import Image

tf.enable_v2_behavior()

BATCH_SIZE = 32
NUM_EPOCHS = 25
NUM_CLASSES = 2  # 2 total classes.
LEARNING_RATE = [0.0001, 0.001, 0.001]

In [2]:
import pandas as pd

annotations = pd.read_csv("annotations.csv")

annotations.tail()

,Image Name,Majority Vote Label,Number of Annotators who Selected SSA (Out of 7),Partition
3147,MHIST_cpn.png,SSA,7,train
3148,MHIST_cfc.png,SSA,7,test
3149,MHIST_cgp.png,SSA,7,test
3150,MHIST_dlf.png,SSA,7,train
3151,MHIST_cks.png,SSA,7,train


In [12]:
annotations.iloc[3151][3]

'train'

In [16]:
from numpy import expand_dims
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.preprocessing.image import ImageDataGenerator
import random

def create_imgs_and_labels(files, note, ratio):
    imgs_arr = []
    labels = []
    # create image data augmentation generator
    datagen = ImageDataGenerator(brightness_range=[0.2,1.0],
                          zoom_range=[0.5,1.0],
                          rotation_range=180)

    for img in files:
      print(img)
      # img_name = img.split("_")[1].strip()
      img_name = img.split('/')[1].strip()
      print(img_name)
    # with open(files, encoding="utf8") as f:
    #   print(f.read())
      
      if annotations.loc[img_name][3] == note:
        img_data = img_to_array(load_img(img, target_size=IMG_DIM))
        label = annotations.loc[img_name][1]
        imgs_arr.append(img_data)
        labels.append(label)
        if ratio == 1:
          continue

        # expand dimension to one sample
        samples = expand_dims(img_data, 0)
        # prepare iterator
        it = datagen.flow(samples, batch_size=1)
        # generate samples and plot
        for i in range(1, ratio):
          if not random.randint(1,10)%5:
            # generate batch of images
            batch = it.next()
            # convert to unsigned integers for viewing
            image = batch[0].astype('uint8')
            imgs_arr.append(image)
            # print(label)
            labels.append(label)
          
    return imgs_arr, labels

In [20]:
import glob
IMG_DIM = (224, 224)

# img_files = "D:\Documents\0_UofT_Courses\ECE1512 Digital Image Processing\ProjectA\Project_A_Supp\Project_A_Supp\mhist_dataset\images\images"
img_files = glob.glob("D:/Documents/0_UofT_Courses/ECE1512 Digital Image Processing/ProjectA/Project_A_Supp/Project_A_Supp/mhist_dataset/images/images/*")
# img_files[0].split('\')[1]
train_imgs, train_labels = create_imgs_and_labels(img_files, "train", 2)
train_imgs = np.array(train_imgs)
train_labels = np.array(train_labels)

test_imgs, test_labels = create_imgs_and_labels(img_files, "test", 1)
test_imgs = np.array(test_imgs)
test_labels = np.array(test_labels)

print('Train dataset shape:', train_imgs.shape, 
      '\tTest dataset shape:', test_imgs.shape)

D:/Documents/0_UofT_Courses/ECE1512 Digital Image Processing/ProjectA/Project_A_Supp/Project_A_Supp/mhist_dataset/images/images\MHIST_aaa.png
Documents


KeyError: 'Documents'

In [ ]:
train_imgs_scaled = train_imgs.astype("float32") / 255.0
test_imgs_scaled = test_imgs.astype("float32") / 255.0

In [ ]:
# encode text category labels
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
le.fit(train_labels)
train_labels_enc = le.transform(train_labels)
test_labels_enc = le.transform(test_labels)
print(train_labels[1000:1020], "\n", train_labels_enc[1000:1020])

In [8]:
from tensorflow.keras.models import Model

# Build CNN teacher.
from tensorflow.keras.applications import ResNet50V2
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions

# Build fully connected student.
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.applications.mobilenet import preprocess_input, decode_predictions


# Build teacher.
def teacher():
  # resnet_model = keras.applications.resnet_v2.ResNet50V2()
  teacher_model = keras.applications.resnet_v2.ResNet50V2(include_top=True, weights='imagenet', input_shape=(224, 224, 3))

  teacher_model.trainable = False
  # for layer in resnet_model.layers:
  #   layer.trainable = False

  x = tf.keras.layers.GlobalAveragePooling2D()(teacher_model.output)
  x = tf.keras.layers.Dense(2, activation="softmax")(x)
  teacher = Model(inputs=teacher_model.input, outputs=x)
  return teacher

# Build student.
def student():
  # mobile_model = keras.applications.mobilenet_v2.MobileNetV2()
  student_model = keras.applications.mobilenet_v2.MobileNetV2(include_top=True, weights="imagenet", input_shape=(224, 224, 3))

  student_model.trainable = False
  # for layer in mobile_model.layers:
  #   layer.trainable = False

  x = tf.keras.layers.GlobalAveragePooling2D()(student_model.output)
  x = tf.keras.layers.Dense(2, activation="softmax")(x)
  student = Model(inputs=student_model.input, outputs=x)
  return student

In [9]:
teacher_model = teacher()
teacher_model.summary()

102869336/102869336 [==============================] - 4s 0us/step


ValueError: Input 0 of layer "global_average_pooling2d" is incompatible with the layer: expected ndim=4, found ndim=2. Full shape received: (None, 1000)